In [ ]:
# if you haven't already set up your environment, install the following:
# pip install fiftyone umap-learn

In [ ]:
import fiftyone as fo
import fiftyone.utils.huggingface as fouh

# Load the dataset from Hugging Face if it's your first time using it

# dataset = fouh.load_from_hub(
#     "Voxel51/Coursera_lecture_dataset_train", 
#     dataset_name="lecture_dataset_train", 
#     persistent=True
#     )

In [ ]:
#because I have the dataset saved locally, I will load it like so
cloned_dataset = fo.load_dataset("lecture_dataset_train_clone")

We'll make use of the outlier detection plugin. You can find more information about that [here](https://github.com/danielgural/outlier_detection).

In [ ]:
from fiftyone import plugins

plugins.download_plugin(
    url_or_gh_repo="https://github.com/danielgural/outlier_detection"
)

plugins.install_plugin_requirements(
    plugin_name="@danielgural/outlier_detection"
)

FiftyOne has a [model zoo](https://docs.voxel51.com/user_guide/model_zoo/models.html) with 100+ models that you can use. Many of which expose embeddings. 

For this lesson, we'll load a simple MobileNetV2 model from the zoo:

In [ ]:
import fiftyone.zoo as foz

model = foz.load_zoo_model(
    "mobilenet-v2-imagenet-torch",
    # device="cuda" # if you're running this on a GPU
    )

We can use this model to compute embeddings for all images in the dataset, note that this could take 10-15 minutes (if you're running on CPU)

In [ ]:
import os 

embeddings = cloned_dataset.compute_embeddings(
    model,
    embeddings_field="mobilenet_v2_embeddings",
    progress=True,
    num_workers=os.cpu_count(),
)

In [ ]:
cloned_dataset

In [ ]:
cloned_dataset.first()

In [ ]:
cloned_dataset.first()['mobilenet_v2_embeddings']

Now, you can perform [dimensionality reduction](https://docs.voxel51.com/tutorials/dimension_reduction.html) so you can visualize the images in the app. 

For this, we'll use the `compute_visualization` method of [FiftyOne Brain](https://docs.voxel51.com/user_guide/brain.html). 


In [ ]:
import fiftyone.brain as fob

fob.compute_visualization(
    cloned_dataset,
    embeddings='mobilenet_v2_embeddings',
    brain_key='mobile_net_viz',
    num_dims=2,
    method='umap'
)

In [ ]:
fo.launch_app(cloned_dataset)

You can also compute embeddings on a patch basis, note you can also compute embeddings and visualizations directly with `compute_visualizations`. Note, we're computing embeddings for each detection, so this process can take a **long** time!

In [ ]:
sampled_dataset = cloned_dataset.take(2000)

In [ ]:
results = fob.compute_visualization(
    samples=sampled_dataset,
    patches_field="ground_truth",
    model=model,
    brain_key="patch_embeddings",
    method="umap",
    num_dims=2,
    num_workers=os.cpu_count(),
    progress=True,
)

In [ ]:
patches_view = sampled_dataset.to_patches("ground_truth", other_fields=True)

In [ ]:
fo.launch_app(patches_view)

Let's explore the outlier detection plugin for the app. 

This plugin uses the Local Outlier Factor (LOF). This is a anomaly detection method that:

1. Measures the local density deviation of a sample compared to its neighbors.

2. Uses k-nearest neighbors to define locality and estimate local density.

3. Identifies outliers by comparing a sample's local density to that of its neighbors.

4. Assigns higher anomaly scores to samples with substantially lower density than their surrounding area.

5. Provides a localized approach to outlier detection, sensitive to the sample's immediate neighborhood context.

You can learn more about LOF [here](https://machinelearninginterview.com/topics/machine-learning/local-outlier-factor-lof/) and [here](https://detectoutliers.com/2023/03/24/using-the-local-outlier-factor-lof-for-unsupervised-anomaly-detection/)

In [ ]:
fo.launch_app(cloned_dataset)

Required reading:

- [Using Image Embeddings](https://docs.voxel51.com/tutorials/image_embeddings.html)

- [How to Visualize Your Data with Dimension Reduction Techniques](https://voxel51.com/blog/how-to-visualize-your-data-with-dimension-reduction-techniques/)

- [Computer Vision Embedding Tips and Tricks](https://medium.com/voxel51/fiftyone-computer-vision-embeddings-tips-and-tricks-mar-31-2023-625b9d9202cd)

- [Finding Outliers in Your Vision Dataset](https://medium.com/voxel51/finding-outliers-in-your-vision-datasets-9bc1d806838c)


If you ever need assistance, have more complex questions, or want to keep in touch, feel free to join the Voxel51 community Discord server [here](https://discord.gg/QAyfnUhfpw)